In [1]:
from syft.core.hooks import TorchHook
from syft.core.hooks import torch
from syft.core.workers import VirtualWorker

hook = TorchHook()
local = hook.local_worker
remote = VirtualWorker(id=1, hook=hook)
local.add_worker(remote)

x = torch.FloatTensor([1,2,3,4,5])
x = x.send(remote)

Hooking into Torch...
Overloading complete.


True

In [2]:
y = x + x

In [3]:
y.get()


  2
  4
  6
  8
 10
[torch.FloatTensor of size 5]

In [4]:
y.id

6405113712

In [5]:
remote._objects

{226556478: [torch.FloatTensor - Locations:[1]],
 6405113712: [torch.FloatTensor - Locations:[1]]}

[1.0, 2.0, 3.0, 4.0, 5.0]

True

True

In [1]:
from syft.core.worker import LocalWorker, BaseWorker

import torch.nn as nn
from torch.autograd import Variable as Var

hook = syft.core.torch_.hook.TorchHook(is_client=True)
local = hook.worker


hook.add_worker(remote.id,remote)

AttributeError: 'TorchHook' object has no attribute 'client_worker'

In [2]:
x = Var(torch.ones(2,2),requires_grad=True).send_(remote)
x2 = Var(torch.ones(2,2),requires_grad=True).send_(remote)

In [3]:
x.backward(x2)

In [4]:
x.get_()

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [5]:
x.grad

In [11]:
remote._objects

{1696728241: Variable containing:[torch.FloatTensor - Locations:[<syft.core.worker.LocalWorker object at 0x103ceafd0>]],
 4378825613: Variable containing:[torch.FloatTensor - Locations:[<syft.core.worker.LocalWorker object at 0x103ceafd0>]]}

In [13]:
remote._objects[4378825613].grad

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]